In [1]:
### System Requirement ###
# At least 32 GB RAM (Cpu)

# Set-up the necessary data for the experiments in playground.ipynb

In [2]:
dataset = "imagenet"
device = 'cuda'
model_name = 'ViT-L-14-336' # 'ViT-H-14'
seed = 0
num_last_layers_ = 4
subset_dim = 10
tot_samples_per_class = 50
dataset_text_name = "top_1500_nouns_5_sentences_imagenet_clean"
datataset_image_name = "imagenet"
algorithm = "svd_data_approx"
path = './datasets'
# Additional params for LLAVA
full_output = False # If we want intermediate hiddn tokens, def False
vision_proj = True # If we want to project in shared space, def True

if model_name == "ViT-H-14":
    pretrained = "laion2B-s32B-b79K"
    precision = "fp32"
elif model_name == "ViT-L-14":
    pretrained = "laion2B-s32B-b82K"
    precision = "fp32"
elif model_name == "ViT-B-16":
    pretrained = "laion2B-s34B-b88K"
    precision = "fp32"
elif model_name == "ViT-B-32":
    pretrained = "laion2B-s34B-b79K"
    precision = "fp32"
elif model_name == "ViT-L-14-336":
    pretrained = "openai"
    precision = "fp16"

In [ ]:
# Retrieve the activations (hooks) for each head in each layer, of a given ViT-model.
# Opt.: Setup a seed and use only a subset of the dataset (Imagenet)
!python -m utils.scripts.compute_activation_values --dataset {dataset} --device {device} --model {model_name} --pretrained {pretrained} --seed {seed} --batch_size 1  --samples_per_class {subset_dim} --tot_samples_per_class {tot_samples_per_class} --quantization {precision} --cache_dir "../cache/" --full_output {full_output} --vision_proj {vision_proj} 

In [ ]:
# Use the previous outputs (i.e. all the activations) to derive the final clip embeddings for the image dataset.
!python -m utils.scripts.compute_images_embedding --dataset {dataset} --model {model_name} --seed {seed}

In [ ]:
# Derive the CLIP-embeddings of the classes' labels of a given dataset (i.e. allow zero-shot classifications)
!python -m utils.scripts.compute_classes_embeddings --dataset {dataset} --device {device} --model {model_name} --pretrained {pretrained} --cache_dir "../cache/"

In [ ]:
# Given a dataset of text compute their CLIP's embeddings.
!python -m utils.scripts.compute_text_embeddings --device {device} --model {model_name} --pretrained {pretrained} --cache_dir "../cache/" --data_path "utils/text_descriptions/{dataset_text_name}.txt"

# Test text explanations

In [ ]:
# Test an algorithm to explain the CLIP-embeddings with text (ours: svd_data_approx, their: text_span)
!python -m utils.scripts.compute_text_explanations --device {device} --model {model_name} --algorithm svd_data_approx --dataset {dataset} --seed {seed} --num_of_last_layers 4 --text_descriptions {dataset_text_name}

In [ ]:
# Print the components with have generated
import json

# Read JSON lines
with open("output_dir/CIFAR10_completeness_top_1500_nouns_5_sentences_imagenet_clean_ViT-L-14-336_algo_svd_data_approx_seed_0_max_text80.jsonl", "r") as json_file:
    for line in json_file:
        entry = json.loads(line)  # Parse each line as a JSON object
        layer = entry["layer"]
        head = entry["head"]
        texts = entry["embeddings_sort"]

        if entry["head"] == -1:
            print(entry.keys())
            print(entry["accuracy"])
        
        print(len(entry["s"]))
        print(f"Layer: {layer}, Head: {head}")
        print("Texts:")
        for text in texts:
            print(text)

In [ ]:
# Go on playground to analyze the results
models = ["ViT-B-32", "ViT-B-16", "ViT-L-14", "ViT-H-14", "ViT-L-14-336"]
algorithms = ["svd_data_approx", "text_span"]
seed = 0
for model in models:
    for algorithm in algorithms:
        with open(f"output_dir/imagenet_completeness_top_1500_nouns_5_sentences_imagenet_clean_ViT-B-32_algo_text_span_seed_0.jsonl", "r") as json_file:

            for line in json_file:
                entry = json.loads(line)  # Parse each line as a JSON object
                layer = entry["layer"]
                head = entry["head"]
                texts = entry["embeddings_sort"]

                if entry["head"] == -1:
                    print(entry.keys())
                    print(entry["accuracy"])